In [44]:
import pandas as pd
import numpy as np
import sqlite3



In [45]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [46]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [47]:
cursor.execute("DROP TABLE IF EXISTS restaurants;")
conn.commit()

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()



In [48]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [49]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")


In [50]:
final_df.shape


(10000, 12)

In [51]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [52]:
from google.colab import files
files.download("final_food_delivery_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [54]:
final_df = final_df.rename(columns={
    "membership": "membership_type",
    "restaurant_name_y": "restaurant_name"
})

# Optional: drop duplicate restaurant_name_x
final_df = final_df.drop(columns=["restaurant_name_x"])


In [55]:
final_df.columns
final_df["membership_type"].value_counts()



,count
membership_type,
Regular,5013
Gold,4987


In [56]:
final_df[final_df["membership_type"]=="Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)



,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [57]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [58]:
user_total = final_df.groupby("user_id")["total_amount"].sum()
user_total[user_total > 1000].count()


np.int64(2544)

In [59]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)



final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-3299981395.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [67]:
print(final_df["order_date"].head())
print(final_df["order_date"].dtype)


0    18-02-2023
1    18-01-2023
2    15-07-2023
3    04-10-2023
4    25-12-2023
Name: order_date, dtype: object
object


In [68]:
# Convert order_date to datetime
final_df["order_date"] = pd.to_datetime(final_df["order_date"], errors="coerce")

# Check how many failed to convert
print("Failed to parse dates:", final_df["order_date"].isna().sum())


Failed to parse dates: 0


/tmp/ipython-input-773084895.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"], errors="coerce")


In [69]:
# Only create quarter for valid dates
final_df["quarter"] = final_df["order_date"].dt.quarter

# Verify
print(final_df[["order_date", "quarter"]].head(10))


  order_date  quarter
0 2023-02-18        1
1 2023-01-18        1
2 2023-07-15        3
3 2023-10-04        4
4 2023-12-25        4
5 2023-07-12        3
6 2023-05-30        2
7 2023-11-07        4
8 2023-12-05        4
9 2023-12-30        4


In [70]:
quarter_revenue = final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
print("Quarter Revenue:\n", quarter_revenue)


Quarter Revenue:
 quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64


In [71]:
# Q1: City with highest total revenue from Gold members
city_gold_revenue = final_df[final_df["membership_type"]=="Gold"].groupby("city")["total_amount"].sum()
print("City Gold Revenue:\n", city_gold_revenue.sort_values(ascending=False))

# Q2: Cuisine with highest average order value
cuisine_aov = final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)
print("\nCuisine AOV:\n", cuisine_aov)

# Q3: Users with total orders > 1000
user_total = final_df.groupby("user_id")["total_amount"].sum()
users_over_1000 = user_total[user_total > 1000].count()
print("\nUsers with orders >1000:", users_over_1000)

# Q4: Restaurant rating range with highest total revenue
rating_revenue = final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)
print("\nRating Range Revenue:\n", rating_revenue)

# Q5: Gold members avg order value by city
gold_city_avg = final_df[final_df["membership_type"]=="Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False)
print("\nGold Avg Order Value by City:\n", gold_city_avg)

# Q6: Cuisine with lowest number of distinct restaurants but significant revenue
restaurant_count = final_df.groupby("cuisine")["restaurant_id"].nunique()
revenue = final_df.groupby("cuisine")["total_amount"].sum()
cuisine_analysis = pd.concat([restaurant_count, revenue], axis=1)
print("\nCuisine Analysis (Restaurants vs Revenue):\n", cuisine_analysis)

# Q7: Percentage of orders by Gold members
gold_orders = final_df[final_df["membership_type"]=="Gold"].shape[0]
total_orders = final_df.shape[0]
gold_order_pct = round((gold_orders / total_orders) * 100)
print("\nPercentage of orders by Gold members:", gold_order_pct, "%")

# Q8: Restaurant with highest AOV < 20 orders
restaurant_stats = final_df.groupby("restaurant_name").agg(
    avg_order_value=("total_amount","mean"),
    order_count=("order_id","count")
)
top_restaurant = restaurant_stats[restaurant_stats["order_count"]<20].sort_values("avg_order_value", ascending=False)
print("\nTop restaurant with <20 orders:\n", top_restaurant.head(5))

# Q9: Membership + cuisine combination contributing highest revenue
combo_revenue = final_df.groupby(["membership_type","cuisine"])["total_amount"].sum().sort_values(ascending=False)
print("\nHighest revenue combo:\n", combo_revenue)

# Q10: Quarter with highest revenue
quarter_revenue = final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
print("\nQuarter Revenue:\n", quarter_revenue)


City Gold Revenue:
 city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

Cuisine AOV:
 cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

Users with orders >1000: 2544

Rating Range Revenue:
 rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

Gold Avg Order Value by City:
 city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

Cuisine Analysis (Restaurants vs Revenue):
          restaurant_id  total_amount
cuisine                             
Chinese            120    1930504.65
Indian             126    1971412.58
Italian            126    2024203.80
Mexican            128    2085503.09

Percentage of orders by Gold members: 50 %

Top restaurant with <20 ord

/tmp/ipython-input-2275339371.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


In [72]:
numerical_answers = {
    "Total Gold Orders": gold_orders,
    "Hyderabad Revenue": round(final_df[final_df["city"]=="Hyderabad"]["total_amount"].sum()),
    "Distinct Users": final_df["user_id"].nunique(),
    "Gold Avg Order Value": round(final_df[final_df["membership_type"]=="Gold"]["total_amount"].mean(), 2),
    "Orders with Rating >= 4.5": final_df[final_df["rating"]>=4.5].shape[0],
    "Gold Orders in Top City (Chennai)": final_df[(final_df["membership_type"]=="Gold") & (final_df["city"]=="Chennai")].shape[0]
}
print("\nNumerical Answers:\n", numerical_answers)



Numerical Answers:
 {'Total Gold Orders': 4987, 'Hyderabad Revenue': 1889367, 'Distinct Users': 2883, 'Gold Avg Order Value': np.float64(797.15), 'Orders with Rating >= 4.5': 3374, 'Gold Orders in Top City (Chennai)': 1337}


In [73]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)
